# Introduction

    For this project, I have used topic modeling to deliver marketing and product insights for SPEEDO.

# The Data

    Prof. Julian McAuley at UC-San Diego has graciously let me use his “Amazon Product Data. It contains tons of data     about Amazon products. Specifically, I have leveraged two datasets: (1) meta-data about products and (2) product     reviews. The aforementioned database has reviews on all types of Amazon products, but these datasets are huge         (~80gb).

    To remedy this, I’ve picked to two smaller datasets that only contain (1) meta-data about products that are in       categorized as “Clothing, Shoes & Jewelry” and (2) reviews about products that are in the “Clothing, Shoes &         Jewelry” category.

# Preprocessing Steps:
    Firstly, I extracted the Asins(unique identifier) for all products of SPEEDO from the Metadata of products.

In [1]:
count = 0 
loadedjson = open('meta_Clothing_Shoes_and_Jewelry.json','r')

# loading the json metadata of products 

In [2]:
allproducts = {}    #making a dictionary
listofcategories = {}

for aline in loadedjson:
    count += 1
    if count % 100000 == 0:
        print(count)
    aproduct = eval(aline)
    
    allproducts[aproduct['asin']] = aproduct
    
    for categories in aproduct['categories']:
        for acategory in categories:
            if acategory in listofcategories:
                listofcategories[acategory] += 1
            if acategory not in listofcategories:
                listofcategories[acategory] = 1

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


In [3]:
count = 0 
allspeedoasins = set()

for aproduct in allproducts:
     theproduct = allproducts[aproduct]
     count += 1
     if count % 100000 == 0:
         print(count/1503384)
     for categories in theproduct['categories']:
        for acategory in categories:
            if 'speedo' in acategory.lower():
                allspeedoasins.add(theproduct['asin'])    

0.06651660520532346
0.13303321041064692
0.19954981561597038
0.26606642082129384
0.33258302602661727
0.39909963123194075
0.4656162364372642
0.5321328416425877
0.5986494468479111
0.6651660520532345
0.7316826572585581
0.7981992624638815
0.8647158676692049
0.9312324728745284
0.9977490780798518


In [4]:
print ("length of all products in the data is: ")
print(len(allproducts))

length of all products in the data is: 
1503384


In [5]:
print ("length of Speedo products is: ")
print(len(allspeedoasins))

length of Speedo products is: 
1625


In [6]:
outputfile = open('allasins.txt', 'w')
# Opening a text file 

outputfile.write(','.join(allspeedoasins))
#overwriting the text file by joining allspeedoasins 

outputfile.close()
# Closing the file 

# Step 2:
    Secondly, I created a dictionary of all Speedo product reviews then extracted the reviews from review data and       dumped them into a json file.

In [7]:
loadedjson = open('reviews_Clothing_Shoes_and_Jewelry.json','r')
# loading json data of reviews

In [8]:
allreviews = {}
count = 0

for aline in loadedjson:
    count += 1
    if count % 100000 == 0:
        print(count)
    areview = eval(aline)
    
    theasin = areview['asin']
    thereviewer = areview['reviewerID']
    
    if theasin in allspeedoasins:
        thekey = '%s,%s'%(theasin,thereviewer)
        allreviews[thekey] = areview

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000


In [9]:
print ("Length of reviews of Speedo is: ")
len(allreviews)

Length of reviews of Speedo is: 


9573

In [10]:
import json
        
json.dump(allreviews,open('allspeedoreviews.json','w'))  # allspeedoreviews

allreviews = json.load(open('allspeedoreviews.json','r'))
# This code will open the json file created above 

# Step 3:
    In the next step I have removed the stop-words like “a”, “an” as these words are so common in English sentence that they can be over represented in the results. For this, I have imported nltk(natural language toolkit).

In [11]:
import nltk    
# removing stopwords from the reviews here to fetch better topic terms

nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

stop_words = stopwords.words('english')

stop_words.append('speedo')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adititripathi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 4:
    Here I have defined a function that would load the asins, summary and reviewtext together in a set for further       processing.

In [12]:
texts = set()

def load_texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            print(topicdata[areview].keys())
            reviewtext = topicdata[areview]['reviewText']
            summary = topicdata[areview]['summary']
            asin = topicdata[areview]['asin']
            review = '%s %s %s' %(asin,summary,reviewtext)
            
            texts.add(review)

load_texts(allreviews)

dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', '

dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', '

dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', '

dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', '

dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'])
dict_keys(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', '

# The model I chose for this project is K-means clustering.

    At first I have imported TfidVectorizer from sklearn feature extraction and K means from sklearn cluster.
    K – means initializes with a pre-determined number of clusters which is 22 in my model.
    The maximum iteration which I set in my model is 100000 i.e my model would only iterate the process until 100000     times.
    The next step involves creating an organized list of topics.
    After having the topics, I have create a directory for classification of topic terms thereby extracting the topic     terms from each topic.
    A dictionary entry is created for each topic which is initially empty. The topic terms are used to name the files     and at the end of the loop the file is closed to make sure that nothing could be read and written further in it.


# Model 1

    Basic Model with no additional conditions.

In [13]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   
# Assigning number of topics, it can be changed later analysing the overlaps among topics.  

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))
    

Top terms per cluster
0: shirt b005qxesww sun great
1: shoes water great b0010odjnk
2: pair one months week
3: good quality product fit
4: like fit nice little
5: small size runs suit
6: old year daughter loves
7: swimsuit great daughter fit
8: bag b000791api gear equipment
9: great fit product comfortable
10: shoe water great size
11: love fit great comfortable
12: 34 size fit waist
13: trunks swim fit great
14: suit great bathing swim
15: aerobics water shoes class
16: size ordered big smaller
17: shorts swim great board
18: flip flops flop comfortable
19: feet shoes water beach
20: stars five four three
21: watch band lap time


In [14]:
import os
outfiles = {}

try:
    os.mkdir('output')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    # it extracts the topic terms from each topic term
    outfiles[atopic] = open(os.path.join('output', '_'.join(topic_terms) + '.txt'), 'w')  
    #creating a dictionary entry for a topic, and I am taking those 
    #topic terms and using them to name the files

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        #predictions = model.predict(Y)
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            #here I am making sure by piping in,that I am adding a line break


for n, f in outfiles.items():
    f.close() 

directory already exists


# Most relevant topic terms and their descriptions.

    1: trunks swim fit great - Wide range of Men's shorts are comfortable, fit well & holds color.
    3: aerobics water class shoes -  Water shoes by Speedo which are perfect for water aerobics.
    5: shirt b005qxesww sun great - Speedo rash guard t-shirt provides great fit, sun protection & comes in good quality nylon fabric.
    19: watch band lap time - Speedo Aquacoach watch has short strap opposed to size mentioned on site. It measures every stroke distance & time accurately.
    21: bag b000791api equipment great -  The mesh bag is big enough to store plenty of stuff.

# Model 2

    Negative Reviews - starred less than 2 in 2013   

In [15]:
texts = set()

def load2texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) < 2:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2013":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

load2texts(allreviews)

In [16]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: wide well irritated fit
1: shoes feet water go
2: watch broke counter return
3: suit material size fit
4: shoe water sole foot
5: sizes sizing narrow even
6: shoes uncomfortable never another
7: apart weeks started smell
8: small 34 way size
9: shoes wet put hurt
10: came stitching day pool
11: months apart fell last
12: shoes velcro insole shoe
13: quality bad buy product
14: heel vacation achilles tendon
15: one faded month lasted
16: seller insoles used plan
17: made many shirt vietnam
18: shorts compression told leg
19: thin toe sole unglued
20: size small old fit
21: pair shower comfortable quality


In [17]:
# Classification of topic terms

import os
outfiles = {}

try:
    os.mkdir('output2')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output2', '_'.join(topic_terms) + '.txt'), 'w')  
    
for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
       
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
           


for n, f in outfiles.items():
    f.close()     

directory already exists


# Most relevant topic terms and their descriptions.

    2: wet beware dangerous hurt - Speedo shoes are quite comfortable but slip on wet floors.
    12: sizing chart misleading small - Fabric is stretchy and comfortable, but sizing is off!
    14: narrow brand inch side -  The side edges of brief are too narrow: highly disappointing.
    15: watch buy counter return -  Watch is good but band is ugly.
    21: flip flops toes plastic - The strap is made up of horrible plastic and cause blisters.

# Model 3

    Positive Reviews - starred more than 4 in 2013  

In [18]:
texts = set()

def load3texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) > 4:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2013":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

load3texts(allreviews)

In [19]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: product great comfortable fit
1: bag b000791api gear swim
2: shoe water great fit
3: shoes water great fit
4: suit great bathing fits
5: well comfortable great b001ibiwqy
6: easy water aerobics pool
7: big read reviews ordered
8: good quality price nice
9: trunks swim fit great
10: shorts great swim board
11: like really great comfortable
12: shirt sun protection b005qxet1m
13: swimsuit great daughter fits
14: 34 suit waist size
15: loves old daughter year
16: size ordered fit wear
17: perfect fit one swim
18: feet beach b000c41uda shoes
19: awesome great quality fit
20: little cute looks small
21: love shoes water fit


In [20]:
# Classification of topic terms

import os
outfiles = {}

try:
    os.mkdir('output3')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output3', '_'.join(topic_terms) + '.txt'), 'w') 
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
           


for n, f in outfiles.items():
    f.close()  

directory already exists


# Most relevant topic terms and their descriptions.

    0: brand chose swimmer perfect -   Recommended for everyday swimmer.
    1: great price value fit -  SPEEDO WOMEN'S SURFWALKER PRO - A great product. Costed a few more dollars, but well worth the price.
    4: quality nice good little -  Speedo Men's Solid Jammer The material is great and very sleek design.
    7: shoes water great comfortable -  GREAT FOR NON-SLIP! These shoes are perfect for cave tubing.
    13: old year loves daughter -   Baby girl products are adorable and secure.
    

# Model 4

    Negative Reviews - starred less than 2 in 2014 

In [21]:
texts = set()

def load4texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) < 2:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2014":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

load4texts(allreviews)

In [22]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: color swimming b009t2oafg ands
1: size small ordered need
2: shoes seam uncomfortable see
3: watch less stopped working
4: suit lining like size
5: shoe size usually disappointed
6: shoe pool well tread
7: short shorts b0047e59lu 8217
8: pair poor quality years
9: compression large small maybe
10: fit small tight runs
11: received wanted days looks
12: water watch tight buttons
13: 34 black wear flops
14: suit faded thought got
15: shoes horrible water minutes
16: broken sent replacement use
17: waist small bigger anyone
18: seams inside daughter rough
19: apart b001ibiwqy came foot
20: big support fabric send
21: wears quality poor material


In [23]:
import os
outfiles = {}

try:
    os.mkdir('output4')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output4', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()     
               

directory already exists


# Most relevant topic terms and their descriptions.

    2: pair apart poor quality - Non-Standard sizes and dense tight fabric – Speedo jammers
    7: watch water horrible using - Speedo watches have technical issues, stops working in few days of use.
    20: tight strange fit boring - Inappropriate range of size – XL is not a regular XL.
    21: strap apart b001ibiwqy came -  Disaster Strap! Men’s pool slide.
    

# Model 5

    Positive Reviews - starred more than 4 in 2014 

In [24]:
texts = set()

def load5texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            if 'overall' in topicdata[areview]:
                if int(topicdata[areview]['overall']) > 4:
                    reviewdate = topicdata[areview]['reviewTime']
                    datenocomma = reviewdate.replace(",", "")
                    mmddyy = datenocomma.split(' ')
                    if mmddyy[2] == "2014":
                        summary = topicdata[areview]['summary']
                        reviewtext = topicdata[areview]['reviewText']
                        asin = topicdata[areview]['asin']
                        review = '%s %s %s' %(asin,summary,reviewtext)
                        texts.add(review)

load5texts(allreviews)

In [25]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: top suit long little
1: loves recommend comfortable would
2: suit bathing great fits
3: five stars great good
4: shirt b005qxesww great swim
5: small trunks size great
6: great fit colors product
7: feet easy water shoes
8: love swimsuit great fits
9: bag b000791api gear great
10: fit like perfect wear
11: shoes water b0010odjnk aerobics
12: daughter loves suit perfect
13: shorts great length swim
14: would size prefer ordered
15: old year suit fits
16: shoes water great fit
17: shoe water great b0010odjnk
18: sun protection b005qxet1m shirt
19: good quality fit expected
20: 34 fit size perfect
21: size smaller 10 great


In [26]:
import os
outfiles = {}

try:
    os.mkdir('output5')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output5', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()  

directory already exists


# Most relevant topic terms and their descriptions.

    3: good quality fit well - Great result: for wide variety of footwear.
    13: excellent beach walk rocks - Great for comfortable beachcoming: Speedo Surfwalkers
    14: suit great bathing fits -  A wide variety of Men’s swimsuit tend to fit comfortably.
    21: looking well great made -  Brief swimsuit: snug fitting.
 

# Model 6

    Reviews during summer 2013(June - August)

In [27]:
texts = set()

def load6texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            reviewdate = topicdata[areview]['reviewTime']
            datenocomma = reviewdate.replace(",", "")
            mmddyy = datenocomma.split(' ')
            if (int(mmddyy[0])>5 and int(mmddyy[0])<9):
                summary = topicdata[areview]['summary']
                reviewtext = topicdata[areview]['reviewText']
                asin = topicdata[areview]['asin']
                review = '%s %s %s' %(asin,summary,reviewtext)
                texts.add(review)

load6texts(allreviews)

In [28]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: bag b000791api gear mesh
1: shoes water great comfortable
2: large medium runs size
3: cute shoes little wear
4: small size runs suit
5: 34 size fit suit
6: shoes feet water shoe
7: love great fit one
8: size big ordered fit
9: shirt b005qxesww great swim
10: swim trunks great fit
11: swimsuit nice great like
12: shorts great swim like
13: loves daughter old fits
14: stars five four great
15: b005qxet1m sun protection great
16: watch band lap laps
17: tight fit size waist
18: shoe water great size
19: suit bathing great suits
20: good quality fit fits
21: pool beach great shoes


In [29]:
import os
outfiles = {}

try:
    os.mkdir('output6')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output6', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()   

directory already exists


# Most relevant topic terms and their descriptions.

    3: 34 suit fit waist - Great jammer! Snug where It's supposed to be, comfortable where you want it to be.
    14: bathing suit suits great -  Toddler bathing suit: comfortable and very good in quality.
    17: shirt great b005qxesww sun -  Comfortable and light weight beach shirt.
    21: shoe water great good -  Speedo Women's Surfwalker Pro: Great Purchase

# Model 7

    Reviews during summer 2014(June - August)

In [30]:
texts = set()

def load7texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            reviewdate = topicdata[areview]['reviewTime']
            datenocomma = reviewdate.replace(",", "")
            mmddyy = datenocomma.split(' ')
            if (int(mmddyy[0])>5 and int(mmddyy[0])<9):
                summary = topicdata[areview]['summary']
                reviewtext = topicdata[areview]['reviewText']
                asin = topicdata[areview]['asin']
                review = '%s %s %s' %(asin,summary,reviewtext)
                texts.add(review)

load7texts(allreviews)

In [31]:
documents = list(texts)

vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(documents)

true_k = 22   # Assigning number of topics

model = KMeans(n_clusters = true_k,
               max_iter = 100000)
model.fit(X)

print("Top terms per cluster")

order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' %(i, ' '. join(topic_terms)))

Top terms per cluster
0: trunks swim dry great
1: great fit comfortable one
2: shorts great swim like
3: perfect fit suit great
4: shoe water great size
5: size ordered fit wear
6: good quality product fit
7: shirt great b005qxesww sun
8: old year loves daughter
9: watch band lap laps
10: suit great bathing fits
11: swimsuit 34 nice like
12: five stars great good
13: big large size medium
14: tight three stars fit
15: shoes water great feet
16: light tee rashguard weight
17: bag b000791api gear mesh
18: apart months use years
19: four stars b00eymys5s shirt
20: pool great comfortable feet
21: small size runs suit


In [32]:
import os
outfiles = {}

try:
    os.mkdir('output7')
    
except OSError:  
    print("directory already exists")

else:
    print ("Successfully created the directory")

for atopic in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[atopic, :4]] 
    outfiles[atopic] = open(os.path.join('output7', '_'.join(topic_terms) + '.txt'), 'w')  
    

for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        reviewwithmetadata = "%s %s %s" % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([reviewwithmetadata])
        
        
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % reviewwithmetadata)
            


for n, f in outfiles.items():
    f.close()             
        

directory already exists


# Most relevant topic terms and their descriptions.

    1: watch band lap laps -  Watch is waterproof but has ugly band and uneasy to wear.
    6: shirt great sun b005qxesww - Shirt which provides awesome sun protection at the beach.
    17: shoes water great comfortable -  Speedo Women's Surfwalker Pro: Quality water shoes.
    21: 34 suit fit size - Size variations: does not follow the standard range.